In [ ]:
# Note to TA: I am going to submit again to finish this assignment.
#             I am scraping the Indeed website instead of the ones in the homework because it's more useful for me.

In [98]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from pprint import pprint
import datetime

In [99]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.indeed_db
collection = db.items

In [100]:
# URL of page to be scraped
url = 'https://www.indeed.com/jobs?q=data+engineer&l=San+Francisco%2C+CA'

In [101]:
# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [102]:
def extract_job_title_from_result(soup): 
    jobs=[]
    results=soup.find_all(name="div", attrs={"class":"row"})
    for result in results:
        for a in result.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return jobs
ls_titles=extract_job_title_from_result(soup)
print(ls_titles)

['Data Engineer', 'Senior Data Engineer', 'Data Engineer Consultant (Python & Hive / Presto)', 'Senior Data Engineer', 'Data Engineer', 'Data Engineer', 'Data Engineer', 'Data Engineer, SF', 'Data Engineer', 'Data Engineer', 'Data Engineer, Stream Platform', 'Data Engineer, Logging Platform', 'Data Engineer', 'Data Engineering Intern', 'Senior Data Engineer', 'React Engineer Hustler at Trustwork']


In [103]:
def extract_company_from_result(soup): 
    companies = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                companies.append(span.text.strip())
    return companies

ls_companies=extract_company_from_result(soup)
print(ls_companies)

['Yodlee', 'Trulia', 'Anectotal LLC', 'LeadGenius', 'Noodle.ai', 'Shipt', 'Segment', 'MLB.com', 'Twitter', 'Esha IT', 'Pinterest', 'Pinterest', 'hiQ LABS', 'Castlight Health', 'Blue Owl', 'Trustwork']


In [104]:
def extract_location_from_result(soup): 
    locations = []
    spans = soup.findAll('span', attrs={'class': 'location'})
    for span in spans:
        locations.append(span.text)
    return(locations)
ls_locations=extract_location_from_result(soup)
print(ls_locations)
print(len(ls_locations))

['Redwood City, CA 94065', 'San Francisco, CA', 'Menlo Park, CA', 'Berkeley, CA 94704', 'San Francisco, CA', 'San Francisco, CA', 'San Francisco, CA 94103 (South Of Market area)', 'San Francisco, CA', 'San Francisco, CA 94103 (South Of Market area)', 'San Francisco, CA', 'San Francisco, CA 94103 (South Of Market area)', 'San Francisco, CA 94103 (South Of Market area)', 'San Francisco, CA 94105 (Financial District area)', 'San Francisco, CA 94105 (Financial District area)', 'San Francisco, CA', 'San Francisco, CA']
16


In [105]:
def extract_summary_from_result(soup): 
  summaries = []
  spans = soup.findAll('span', attrs={'class': 'summary'})
  for span in spans:
    summaries.append(span.text.strip())
  return(summaries)
ls_summary=extract_summary_from_result(soup)
print(ls_summary)

['An ability to analyze data and build data products. The Data Engineer is responsible for creating robust and accessible data sources for our customers in...', '5 Years of proven working experience as a data engineer. Technical expertise regarding data models, database design development, data mining and segmentation...', 'This data engineering role consultant opportunity involved building and maintaining data engineering pipelines, code, data quality checks, third-party...', 'Data sources that have defined schemas) and unstructured data (e.g. Experience with processing large volume of data and production data pipeline....', 'As a Data Engineer, you will collaborate with the Noodle Client Service team, Data Scientists, SW Engineers, and UX Designers, as well as industry-specific...', 'Shipt has a wide variety of data partners and as our data volumes. Data Engineering at Shipt primarily focuses on retailer catalog and general product data for...', 'Own and optimize Segment’s AWS intern

In [106]:
def extract_link_from_result(soup): 
    links=[]
    results=soup.find_all(name="div", attrs={"class":"row"})
    for result in results:
        for a in result.find_all(name="a", attrs={"class":"jobtitle"},href=True):
            links.append("https://www.indeed.com"+a["href"])

    results=soup.find_all(name="h2", attrs={"class":"jobtitle"})
    for result in results:
        for a in result.find_all(name="a",href=True):
            links.append("https://www.indeed.com"+a["href"])
    return links

ls_links=extract_link_from_result(soup)
print(len(ls_links))
pprint(ls_links)

16
['https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0A38a9IVvk5r1TDo7iObPi9660u_wRAOWJAhHUAQ0rFsYKDo_PPgc_1F7ReqEpT1Kub3_SbEm0suqsYfbc7IVdPfNfgdCf3iQU5pacXmdWJ3rmvjj9eU4WcBrJuR6xZrmMQcxLWQ0ciE5HM0UcS8FaiQcJvJzYl8_3dqVnd9N2FS3msXcryUV_zTszRkmdzks1cjJIVY2MKYVx_FB6Ey5wibpHONs9jGnngZxNstrMgMy3sD4jmfzheoIE_O-GByJYS3eaY5Ym17zZKl_M2AhWWzA0MygTvLYtzAORctBbHz6jKw3zK0Bfed-ffUf9jBe9AzQdj0_331d9JMXRAzG5pVEn9ug2glGWBS8pQlzomwwAJxUdXZs9bPYzbgCQQG64cl-cLFBu1xCCMRlaJDVZ7SNO1iG9vuaUZafe_7DrHoLFTDDWG3EiZ3A38ISv4E8Ekx1VvKvWnlClSEruArgFOScJQG1KaGpMli0m66xM3gHCBT0wSBX9uJA92Xz3G7dnvkljtatl-aaMuKdCVq9W2SPEXtGsv1gvXPAJbRnkJDN5tOqC-MtGFn-MqxTONHu4KEGeLsqoDef7k5kZcR4UZ&vjs=3&p=1&sk=&fvj=0',
 'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CZTNjLMuBQEc4UwMDLf68nYWD4vZp_o6pW13OpR78fsUbLF7akhdL4HTVuMU9ZbeK6p2rL4A4W-nNfc3zdHY1wmLw0SfOZ-Cv_RvbZspyyZlGpyyreBIbJ6BJR4f3pkSDKOoykvXxo0ZYCVfgfccVs4bCYqXV_SUifC95Z5f9JIKfr3yRT9Y7AKyDYnU-uK0XJOIm43U7-9Lr8mWMGocqNa8yWCVOu7Igr8RqzJK2i80TydF9Ifs3B2haAw8NkPRgsjDtOPtAKom

In [107]:
# def extract_link_from_result(soup): 
#     links=[]
#     results=soup.find_all(name="h2", attrs={"class":"jobtitle"})
#     for result in results:
#         for a in result.find_all(name="a",href=True):
#             links.append(a["href"])
#     return links
# ls_links=extract_link_from_result(soup)
# print(len(ls_links))
# pprint(ls_links)

In [108]:
# TODO:find the total number of jobs and use splinter to navigate to the next page.

In [109]:
# Create a dictionary that has all the jobs and put the dictionary in a mongoDB
positions=[]
d={}
idx=0
while idx<len(ls_titles):
    d["title"]=ls_titles[idx]
    d["company"]=ls_companies[idx]
    d["location"]=ls_locations[idx]
    d["summary"]=ls_summary[idx]
    d["link"]=ls_links[idx]
    d["Scrape datetime"]=datetime.datetime.utcnow()
    positions.append(d)
    d={}
    idx+=1
pprint(positions)

[{'Scrape datetime': datetime.datetime(2018, 4, 23, 19, 55, 39, 442972),
  'company': 'Yodlee',
  'link': 'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0A38a9IVvk5r1TDo7iObPi9660u_wRAOWJAhHUAQ0rFsYKDo_PPgc_1F7ReqEpT1Kub3_SbEm0suqsYfbc7IVdPfNfgdCf3iQU5pacXmdWJ3rmvjj9eU4WcBrJuR6xZrmMQcxLWQ0ciE5HM0UcS8FaiQcJvJzYl8_3dqVnd9N2FS3msXcryUV_zTszRkmdzks1cjJIVY2MKYVx_FB6Ey5wibpHONs9jGnngZxNstrMgMy3sD4jmfzheoIE_O-GByJYS3eaY5Ym17zZKl_M2AhWWzA0MygTvLYtzAORctBbHz6jKw3zK0Bfed-ffUf9jBe9AzQdj0_331d9JMXRAzG5pVEn9ug2glGWBS8pQlzomwwAJxUdXZs9bPYzbgCQQG64cl-cLFBu1xCCMRlaJDVZ7SNO1iG9vuaUZafe_7DrHoLFTDDWG3EiZ3A38ISv4E8Ekx1VvKvWnlClSEruArgFOScJQG1KaGpMli0m66xM3gHCBT0wSBX9uJA92Xz3G7dnvkljtatl-aaMuKdCVq9W2SPEXtGsv1gvXPAJbRnkJDN5tOqC-MtGFn-MqxTONHu4KEGeLsqoDef7k5kZcR4UZ&vjs=3&p=1&sk=&fvj=0',
  'location': 'Redwood City, CA 94065',
  'summary': 'An ability to analyze data and build data products. The Data '
             'Engineer is responsible for creating robust and accessible data '
             'sources f

In [110]:
db.items.insert_many(positions)

In [67]:
# db.items.insert_many([{"title": "testTitle","location":"testLocation"},{"title": "testTitle2","location":"testLocation2"}])
# # db.test.insert_many([{'x': i} for i in range(2)])
# db.items.delete_one(
#     {'title': 'testTitle2'}
# )

In [111]:
# Verify results:
results = db.items.find()
count=0
try:
    for result in results:
        count+=1
        pprint(str(count)+result['title'])
        pprint(result['company'])
except Exception as e:
    print(e)
print(count)

'1Data Engineer'
'BioCentury Inc.'
'2Senior Data Engineer'
'Blue Owl'
'3Security and Data Integrity Engineer'
'KQED Inc.'
'4Data Engineer Consultant (Python & Hive / Presto)'
'Anectotal LLC'
'5Data Engineer'
'Noodle.ai'
'6Data Engineer'
'Segment'
'7Data Engineer'
'Shipt'
'8Data Engineer, SF'
'MLB.com'
'9Data Engineer'
'Twitter'
'10Data Engineer, Stream Platform'
'Pinterest'
'11Data Engineer, Logging Platform'
'Pinterest'
'12Data Engineer'
'Esha IT'
'13Data Engineering Intern'
'Castlight Health'
'14Machine Learning Engineer, Notification Data'
'Pinterest'
'15Data Engineer'
'Yodlee'
'16Senior Data Engineer'
'Trulia'
'17Data Engineer'
'Yodlee'
'18Senior Data Engineer'
'Trulia'
'19Data Engineer Consultant (Python & Hive / Presto)'
'Anectotal LLC'
'20Senior Data Engineer'
'LeadGenius'
'21Data Engineer'
'Noodle.ai'
'22Data Engineer'
'Shipt'
'23Data Engineer'
'Segment'
'24Data Engineer, SF'
'MLB.com'
'25Data Engineer'
'Twitter'
'26Data Engineer'
'Esha IT'
'27Data Engineer, Stream Platform'
'P